In [126]:
import MySQLdb
import pandas
import itertools
import numpy as np
import bm25fe
import pickle

In [112]:
conn = MySQLdb.connect(host='10.117.8.41', port=3306, user='root', passwd='vmware', db='bugfeature')
cur = conn.cursor()

sql = '''SELECT dup_of, dup
FROM dups_cpdplatform left join bugs_cpdplatform as b1 on dups_cpdplatform.dup=b1.bug_id left join bugs_cpdplatform as b2 on dups_cpdplatform.dup_of=b2.bug_id where b1.long_desc != '' and b2.long_desc != '' '''

cur.execute(sql)
dups = cur.fetchall()

dups_train = dups[:128]
print len(dups_train)

dupset = set(itertools.chain(*dups))

128


In [113]:
conn = MySQLdb.connect(host='10.117.8.41', port=3306, user='root', passwd='vmware', db='bugfeature')
cur = conn.cursor()

sql = '''SELECT bug_id, short_desc, long_desc
FROM bugs_cpdplatform'''

# bugs = pandas.io.sql.read_sql(sql, conn).set_index(['bug_id'])
bugs = pandas.io.sql.read_sql(sql, conn)

# bugs_filtered = bugs[(not bugs['bug_id'] in dupset) & (len(bugs['long_desc'])>0)]
criterion1 = bugs['bug_id'].map(lambda x:  not x in dupset)
criterion2 = bugs['long_desc'].map(lambda x:  len(x)>0)
bugs_filtered = bugs[ criterion1 & criterion2]

In [114]:
train = []
bugs_filtered.sample(random_state=0)
for item in dups_train:
    train += list(bugs_filtered.sample(n=10)['bug_id'].map(lambda x: (item[0], item[1], x)))

print len(train)

1280


In [115]:
bugset = set(itertools.chain(*train))
criterion = bugs['bug_id'].map(lambda x: x in bugset)
bugs_train = bugs[criterion]
n = bugs_train.shape[0]
texts = list(bugs_train['short_desc']) + list(bugs_train['long_desc'])

In [116]:
# print n
# print len(texts)
# print len(bugset)
# print train
# print bugs_filtered.shape
# print criterion
# print bugset
# print bugs.shape
# print type(criterion1)

In [117]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(analyzer = "word",ngram_range=(1, 1), tokenizer = None, preprocessor = None,
                             stop_words = None, lowercase = False, 
                             max_df = 0.9, min_df = 10, max_features = 100000)

vectors = vectorizer.fit_transform(texts).toarray()
vectors_sd = np.array(vectors[:n])
vectors_ld = np.array(vectors[n:])

bug_feature = pandas.DataFrame({'bug_id':bugs_train['bug_id'], 'short_desc':list(vectors_sd), 'long_desc':list(vectors_ld)}).set_index(['bug_id'])

appearance = (vectors_sd + vectors_ld)>0
df = appearance.sum(0)
idf = np.log(n/df)
avgfl = np.array([vectors_sd.sum(1).mean(), vectors_ld.sum(1).mean()])

In [118]:
# print type(vectors[:n])
# print vectors[0]
# print len(vectorizer.vocabulary_)
# print bug_feature.iloc[0,1]
# print list(vectors_sd)[0]
# print vectors_sd.sum(1).mean()

In [119]:
def costfunc (params, train, bug_feature, idf, avgfl):
    scorer = bm25fe.bm25fe(K1=params[0], d_B=(params[1], params[2]), d_W = (params[3], params[4]), K3=params[5], q_B=(params[6], params[7]), q_W=(params[8], params[9]))
    cost = 0
    for item in train:
        cost += np.log(1+np.exp(scorer.score(idf, avgfl, [bug_feature.loc[item[2],'short_desc'], bug_feature.loc[item[2],'long_desc']],
                                            [bug_feature.loc[item[0],'short_desc'], bug_feature.loc[item[0],'long_desc']])-
                                scorer.score(idf, avgfl, [bug_feature.loc[item[1],'short_desc'], bug_feature.loc[item[1],'long_desc']],
                                            [bug_feature.loc[item[0],'short_desc'], bug_feature.loc[item[0],'long_desc']])))
    return cost

In [120]:
def costfunc_der (params, train, bug_feature, idf, avgfl):
    scorer = bm25fe.bm25fe(K1=params[0], d_B=(params[1], params[2]), d_W = (params[3], params[4]), K3=params[5], q_B=(params[6], params[7]), q_W=(params[8], params[9]))
    der = np.zeros(params.size)
    for item in train:
        differ = (scorer.score(idf, avgfl, [bug_feature.loc[item[2],'short_desc'], bug_feature.loc[item[2],'long_desc']],
                               [bug_feature.loc[item[0],'short_desc'], bug_feature.loc[item[0],'long_desc']])-
                  scorer.score(idf, avgfl, [bug_feature.loc[item[1],'short_desc'], bug_feature.loc[item[1],'long_desc']],
                               [bug_feature.loc[item[0],'short_desc'], bug_feature.loc[item[0],'long_desc']]))
        der += np.exp(differ)/(1+np.exp(differ))*(scorer.derivative(idf, avgfl, [bug_feature.loc[item[2],'short_desc'], bug_feature.loc[item[2],'long_desc']],
                                                             [bug_feature.loc[item[0],'short_desc'], bug_feature.loc[item[0],'long_desc']])-
                                                  scorer.derivative(idf, avgfl, [bug_feature.loc[item[1],'short_desc'], bug_feature.loc[item[1],'long_desc']],
                                                               [bug_feature.loc[item[0],'short_desc'], bug_feature.loc[item[0],'long_desc']]))
    return der

In [121]:
# bm25fe = reload(bm25fe)

In [122]:
from scipy import optimize
params0 = [1.2, 0.75, 0.75, 0.5, 0.5, 1.2, 0.75, 0.75, 0.5, 0.5]
bounds = ((0, None), (0, 1), (0, 1), (0, 1), (0, 1), (0, None), (0, 1), (0, 1), (0, 1), (0, 1))

result = optimize.minimize(costfunc, params0, args=(train, bug_feature, idf, avgfl), jac=costfunc_der, bounds=bounds)

print result

  status: 2
 success: False
    nfev: 215
     fun: nan
       x: array([  1.09256319e+02,   7.48227365e-01,   1.00000000e+00,
         6.49770392e-01,   1.16469720e-02,   5.94334749e-03,
         4.52067286e-01,   6.87237714e-02,   1.00000000e+00,
         6.78361624e-03])
 message: 'ABNORMAL_TERMINATION_IN_LNSRCH'
     jac: array([ -1.85300073e-03,  -2.18933100e+00,  -2.61350943e+03,
        -3.28470353e+00,  -5.36421347e+01,   4.68261575e+01,
         9.17508896e-03,  -4.33347455e-01,  -4.58505307e-02,
        -3.68698798e+01])
     nit: 107


In [124]:
print type(result.x)

<type 'numpy.ndarray'>


In [127]:
with open('params.txt', 'wb') as f:
    pickle.dump(result.x, f)
    f.close()

In [90]:
# print train
# print str(915183) in bugset
# print bugset
# print len(bugset)
# print bugs_train.shape

False
3498
(3498, 3)
